In [1]:
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
import time
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from collections import Counter
from IPython.display import clear_output

from scripts import BpeTokenizer, Model, Trainer, Collator, MyDataset, generate

# Загружаем данные

In [2]:
df = pd.read_csv('data/dataset.csv')
train_texts = df['text'][:-1024].tolist()
eval_texts = df['text'][-1024:].tolist()

# Инициализируем и обучаем токенизатор

In [3]:
tokenizer = BpeTokenizer()

In [4]:
tokenizer.train(train_texts[:2048], max_vocab=2048)

pair=(277, 338), freq=52: 100%|██████████| 1789/1789 [02:33<00:00, 11.66it/s]  


# Создаем датасеты и Collator

In [5]:
train_dataset = MyDataset(train_texts, tokenizer, max_length=128)
eval_dataset = MyDataset(eval_texts, tokenizer, max_length=128)
collator = Collator(tokenizer.pad_token_id)

100%|██████████| 1024/1024 [00:35<00:00, 29.16it/s]


# Создаем модель

In [6]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [7]:
model = Model(tokenizer.get_vocab_size(), emb_size=128, hidden_size=256, num_layers=2, dropout=0.1)

# Создаем Trainer и запускаем обучение

In [8]:
train_loader = DataLoader(
            train_dataset,
            batch_size=32,
            shuffle=True,
            drop_last=True,
            collate_fn=collator
        )

In [9]:
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    n_epochs=8,
    train_batch_size=32,
    eval_batch_size=32,
    eval_steps=64,
    collator=collator,
    lr=1e-2,
    ignore_index=tokenizer.pad_token_id
)

In [10]:
trainer.train()

  0%|          | 0/4096 [00:00<?, ?it/s]e:\llm-course-2024-autumn\Homework\01\scripts\trainer.py:113: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3575.)
  x = torch.stack([ids.T[i].T for i in range(0, len(ids[0]) - 1)])
epoch=0.126953125, loss=5.516380786895752:   2%|▏         | 65/4096 [00:13<57:38,  1.17it/s] 

epoch=0.125, eval_loss=5.4764887392520905


epoch=0.251953125, loss=4.847661972045898:   3%|▎         | 129/4096 [00:26<55:11,  1.20it/s] 

epoch=0.25, eval_loss=4.807385876774788


epoch=0.376953125, loss=4.625861644744873:   5%|▍         | 193/4096 [00:40<54:20,  1.20it/s] 

...

epoch=7.75, eval_loss=4.280873671174049


epoch=7.876953125, loss=4.265037536621094:  98%|█████████▊| 4033/4096 [13:56<00:52,  1.20it/s] 

epoch=7.875, eval_loss=4.2777193784713745


epoch=8.0, loss=4.255505084991455: 100%|██████████| 4096/4096 [14:09<00:00,  4.82it/s]        

epoch=8.0, eval_loss=4.277573078870773


# Оцениваем качество и проверяем жадную и случайную генерацию

In [11]:
trainer.evaluate()

4.277573078870773

In [12]:
generate(model, tokenizer, temperature=0)

'Овнам стоит быть внимательны и наиболее продуктивный день, особенно важное время для того, чтобы вы способны стать ждет день, особенно важное время для того, чтобы вы способны стать ждет день, особенно важное время для того, чтобы вы способны стать ждет день, особенно важное время для того, чтобы вы способны стать ждет день, особенно важное время для того, чтобы вы способны стать ждет день, особенно важное время для того, чтобы вы способны стать ждет день, особенно важное время для того, чтобы вы способны стать ждет день, особенно важное время для того, чтобы вы способны стать ждет день, особенно важное время для того, чтобы вы способны стать ждет день, особенно важное время для того, чтобы вы способны стать ждет день, особенно важное время для того, чтобы вы способны стать ждет день, особенно важное время для того, чтобы вы способны стать ждет день, особенно важное время для того, чтобы вы способны стать ждет день, особенно важное время для того, чтобы вы способны стать ждет день, ос

In [13]:
generate(model, tokenizer, temperature=0.5, top_k=20)

'Звезды советуют неизм не стоит игно в течение дня у вас есть возможность для Скорпионам стоит внимательны и на день, двёнет вам стоит быть черены накладить наслабиться, особенно ссорылаширадалиментов. В конце дня вам нескольку и назмешняяхотите удастся не хоборот, природную работузды советуют не судов. Всобные или остроить накладывают, что вполагадцами, ссорами или судьба. Это удачный момент для посредит вам наследствузвяхотят вам не таких целомнения долгожданов воварические способна в это время вы не лучший момент для того, репискуса, которых выдержки в течение дня вы можете не ссорыльзысколько вторую, пищи, сы или с партнерами и сотрудничества, капротивных и развитие настройством или налашихсят период для себя визит относиться, тонтуальное мероприятия, вы неформальное участие возможны и о себе отправлением и морьбы или от вас насущных и дображное, что вы рискованный момент для завершите, что выражать свои личные в любовьтесь ктошний деньгиозясь поэтому несобнова кого вовсущение в эт